# My Notes 

## Research papers of interest
These papers are stored at the Arxiv website, they are mostly have not been through peer reviews because they are so new. On the other hand, they are follow up and performed by others researchers and interested users on the internet, if there are problems these will be report almost instantly on the social network.

[diffedit](https://arxiv.org/abs/2210.11427)  
[Progressive Distillation for Fast Sampling of Diffusion Models](https://arxiv.org/abs/2202.00512)  
[On Distillation of Guided Diffusion Models](https://arxiv.org/abs/2210.03142)  
[Imagic: Text-Based Real Image Editing with Diffusion Models](https://arxiv.org/abs/2210.09276)  



## Eistein Summation

A technique for convenient and simplify the writing of syntax for many operation in linear algebra

- Matrix multiplication
- Element-wise matrix operation
- Permutation of matrix
- Dot product of matrix
- Outer product of matrix
- Summation of matrix
- Batch multiplication of matrix (permute input to match function calls ordering)

It's also speedup some of the above operation especially operation that can be combined into single call

The eisum is a build-in feature of most Machine learning frameworks, e.g. Pytorch, Tensorflow, Flux...

How does it work?

Here is the matrix multiplication  
$$ M_{ij} = \sum{A_{ik}B_{kj}} = A_{ik}B_{kj}$$

In [1]:
import numpy as np

In [2]:
A = np.random.rand(3,5)
B = np.random.rand(5,2)
M = np.empty((3,2))

A.shape,B.shape,M.shape

((3, 5), (5, 2), (3, 2))

In [3]:
A,B,M

(array([[0.29744053, 0.69604914, 0.67117175, 0.26387973, 0.09389456],
        [0.42746273, 0.09049403, 0.90969471, 0.29861237, 0.2264451 ],
        [0.55364468, 0.50495913, 0.65850403, 0.06964918, 0.31421708]]),
 array([[0.78620971, 0.20917867],
        [0.20291897, 0.5199813 ],
        [0.48062955, 0.75832942],
        [0.31415192, 0.1330993 ],
        [0.21116944, 0.61648433]]),
 array([[1.13610772e-313, 0.00000000e+000],
        [0.00000000e+000, 0.00000000e+000],
        [0.00000000e+000, 0.00000000e+000]]))

In [4]:
np.set_printoptions(precision=2, linewidth=140)

### Code:  
Coding as loop of the matrix multiplication above   
where the row is i and column is j, and k is the inner dimension of both matrix that must be equal and this index will be summed and disappeared by the operation of matrix rule

In [5]:
for i in range(3):
    for j in range(2):
        total = 0
        for k in range(5):
            total += A[i,k] * B[k,j]
        M[i,j] = total

print(f'the matrix is: {M}')

the matrix is: [[0.8  1.03]
 [0.93 1.01]
 [0.94 1.08]]


Using the Eisum method

In [6]:

# the i and j are free index
# the k is sum index since it will be summed away after the operation
M1 = np.einsum('ik,kj->ij',A,B)

print(f'the matix is : {M1}')

the matix is : [[0.8  1.03]
 [0.93 1.01]
 [0.94 1.08]]


### Dot product
Vector or matrix multiplicaton $u \cdot v$  that result of scalar value  

### Outer product 
vector multiply by vector that result in a matrix $u \otimes v$  

Example 2
- Using the free index in the output  
- No summation index  

In [7]:
# example 2
D = np.random.rand(5)
E = np.random.rand(3)
out = np.einsum('i,j->ij',D,E)
print(f'the matrix is: \n{out}')

the matrix is: 
[[0.1  0.09 0.37]
 [0.17 0.15 0.64]
 [0.12 0.11 0.45]
 [0.16 0.14 0.57]
 [0.02 0.02 0.07]]


In [8]:
# loop version
for i in range(5):
    for j in range(3):
        total = 0
        total += D[i] * E[j]
        out[i,j] = total
        
print(f'the matrix is: \n{out}')

the matrix is: 
[[0.1  0.09 0.37]
 [0.17 0.15 0.64]
 [0.12 0.11 0.45]
 [0.16 0.14 0.57]
 [0.02 0.02 0.07]]


## The Eisum Rules

The free indices: 
- The index that specify the output

The Summation index  
- All other indices that appear in the input argument but not show up in the output

The General rules:
1. Same index in the a different input argument indicate that these indices will be multiplied and the product are outputed
``` python
    M = np.einsum('ik,kj->ij',A,B)
```
2. Omitting index indicate the index will be summed together
``` python
    X = np.ones(3)
    Y = np.einsum('i->',X)
```
3. The unsummed indices may return in any order
``` python
    D = np.ones((5,4,3))
    E = np.einsum('ijk->kji',D)
```

Operation that benefit from Einsum
1. Permutation of Tensors
2. Summation
3. Column sum
4. Row sum
5. Matrix-Vector multiplication
6. Matrix-Matrix multiplication
7. Dot Product the first row with first row of a matrix
8. Dot product with matrix (multiplication and add)
9. Element-wise multiplication (Hadamard Product) (multiplication no add)
10. Outer Product
11. Batch matrix multiplicaton e.g. a = 3,2,6 and b = 3,6,3
    - want to multiply the matrix of 2x6 with 6x3 matrix
    - these matrix must follow the multiplication rule
    - the first number  is the batch size they must match, but not count as index 
    - the torch.bmm function will do the same thing
12. Matrix diagonal  
    - return the only the diagonal value of the matrix
13. Matrix Trace
    - summing the value of the diagonal of a matrix

In [9]:
import torch
import numpy as np

torch.set_printoptions(precision=2, linewidth=140)

X = torch.rand((2,3))
X

tensor([[0.44, 0.49, 0.98],
        [0.75, 0.81, 0.48]])

1. Transpose
Switching the index of a matrix

In [10]:
# 1. transpose

c_ntp = np.transpose(X)
c_tp = torch.transpose(X,0,1)
cein = torch.einsum('ij->ji',X)
print(f'regular: {c_tp}\n numpy: {c_ntp} \neisum: {cein}')

regular: tensor([[0.44, 0.75],
        [0.49, 0.81],
        [0.98, 0.48]])
 numpy: tensor([[0.44, 0.75],
        [0.49, 0.81],
        [0.98, 0.48]]) 
eisum: tensor([[0.44, 0.75],
        [0.49, 0.81],
        [0.98, 0.48]])


2. Matrix summation  
Summing all value in the matrix that result in a scalar value

In [11]:
#2. Summation
cma = torch.sum(X)
cein = torch.einsum('ij->',X)

print(f'regular: {cma} \neisum: {cein}')

regular: 3.9641716480255127 
eisum: 3.9641716480255127


In [12]:
X

tensor([[0.44, 0.49, 0.98],
        [0.75, 0.81, 0.48]])

Row sum (Top down)

Add all values from each row together along the column

In [13]:
#4 row summation
# sum by columns
rows = torch.sum(X,dim=0)
cein = torch.einsum('ij->j',X)

print(f'regular: {rows} \neisum: {cein}')

regular: tensor([1.19, 1.30, 1.47]) 
eisum: tensor([1.19, 1.30, 1.47])


Column sum (Left to right)

Add all value from each column together along the row

In [14]:
#3 Column summation
# sum by rows
c_col = torch.sum(X,dim=1)
cein = torch.einsum('ij->i',X)

print(f'regular: {c_col} \neisum: {cein}')

regular: tensor([1.91, 2.05]) 
eisum: tensor([1.91, 2.05])


5 matrix-vector multiplication  
This a non equal dimension multiplication which in Python use broadcasting to padded (duplicate) the vector to have equal size with the matrix before do multiplication

In [15]:
#5 matrix-vector multiplication
L = torch.rand((1,3))
M = torch.rand((3,))

cmm = torch.matmul(L,M)
cein = torch.einsum('ij,j->i',L,M)
print(f'regular: {cmm} \neisum: {cein}')


regular: tensor([0.97]) 
eisum: tensor([0.97])


6 matrix-matrix multiplication  
This standard matrix to matrix multiplication which result in another matrix

In [16]:
#6 matrix-matrix multiplication
# torch.einsum('ij,kj->ik',M,M)

a = torch.ones((3,2))
b = torch.ones((2,3))
cmm = torch.matmul(a,b)
cein = torch.einsum('ij,jl->il',a,b)
print(f'regular: {cmm} \neisum: {cein}')


regular: tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]]) 
eisum: tensor([[2., 2., 2.],
        [2., 2., 2.],
        [2., 2., 2.]])


In [17]:
N = torch.rand((3,3))
M = torch.rand((2,3))
torch.einsum('ij,kj->ik',N,M)

tensor([[0.27, 0.87],
        [0.41, 1.24],
        [0.49, 1.50]])

Dot product

This a matrix multiplication that result in a scalar value. It's usually called multiply add.  
since after multiply the row to the column then the sum operation is carry out result as a scalar

In [18]:
# dot product of a matrix (multiply + add)
#torch.einsum('ij,ij->',N,N)

# c = torch.tensor([2,3])
# d = torch.tensor([2,1])
c = torch.rand((3))
d = torch.rand((3))

c_dot = torch.dot(c,d)
cein = torch.einsum('i,i->',c,d)

print(f'c: {c}, c: {c.shape}')
print(f'c_dot: {c_dot}')
print(f'regular: {c_dot} \n  eisum: {cein}')

c: tensor([0.03, 0.54, 0.61]), c: torch.Size([3])
c_dot: 0.5557287931442261
regular: 0.5557287931442261 
  eisum: 0.5557287931442261


In [19]:
# dot product of only the first row of a matrix with first row of a matrix
torch.einsum('i,i->',N[0],N[0])

tensor(0.54)

Element wise multiplication (multiply only)  
This is a normal matrix multiplication which different from multiply add or dot product

In [20]:
# element wise multiplication (multiply only)
# torch.einsum('ij,ij->ij',N,N)

c = torch.randn((3,2))
d = torch.randn((3,2))
cmm = c * d
cein = torch.einsum('ij,ij->ij',c,d)
print(f'regular: {cmm} \n  eisum: {cein}')

regular: tensor([[-0.74,  1.05],
        [-0.21, -0.99],
        [ 1.41,  0.00]]) 
  eisum: tensor([[-0.74,  1.05],
        [-0.21, -0.99],
        [ 1.41,  0.00]])


Outer Product

Multiply vector of different size to get a matrix as output
In eisum must use different letter to represent size different


In [21]:
# outer product
x = torch.rand(3)
y = torch.rand(5)
print(f'x: {x}, x: {x.shape}')
print(f'y: {y}, y: {y.shape}')

c_outer = torch.outer(x,y)
cein = torch.einsum('i,j->ij',x,y)
print(f'regular: {c_outer} \n  eisum: {cein}')

x: tensor([0.37, 0.62, 0.73]), x: torch.Size([3])
y: tensor([0.06, 0.51, 0.01, 0.66, 0.49]), y: torch.Size([5])
regular: tensor([[0.02, 0.19, 0.00, 0.24, 0.18],
        [0.04, 0.31, 0.01, 0.41, 0.30],
        [0.04, 0.37, 0.01, 0.48, 0.36]]) 
  eisum: tensor([[0.02, 0.19, 0.00, 0.24, 0.18],
        [0.04, 0.31, 0.01, 0.41, 0.30],
        [0.04, 0.37, 0.01, 0.48, 0.36]])


Batch matrix multiplication  
Multiply matrix by the set of n, where n is batch size  

want to multiply the matrix of 2x6 with 6x3 matrix  
the first number is the batch size must match but not count as index so i is ignore  
the mxn * nxp must match with n

In [22]:
# batch matrix multiplicaton
# want to multiply the matrix of 2x6 with 6x3 matrix
# the first number is the batch size must match but not count as index so i is ignore
# the mxn * nxp must match with n
R = torch.rand(3,2,6)
S = torch.rand(3,6,3)
cmn = np.matmul(R,S)
cmm = torch.matmul(R,S)

cein = torch.einsum('ijk,ikl->ijl',R,S)

print(f'regular: {cmm}\n numpy: {cmn} \n  eisum: {cein}')

regular: tensor([[[1.33, 1.04, 0.80],
         [1.46, 1.10, 0.66]],

        [[1.42, 1.49, 2.23],
         [0.99, 1.02, 1.58]],

        [[1.32, 2.04, 1.41],
         [0.82, 1.90, 1.11]]])
 numpy: tensor([[[1.33, 1.04, 0.80],
         [1.46, 1.10, 0.66]],

        [[1.42, 1.49, 2.23],
         [0.99, 1.02, 1.58]],

        [[1.32, 2.04, 1.41],
         [0.82, 1.90, 1.11]]]) 
  eisum: tensor([[[1.33, 1.04, 0.80],
         [1.46, 1.10, 0.66]],

        [[1.42, 1.49, 2.23],
         [0.99, 1.02, 1.58]],

        [[1.32, 2.04, 1.41],
         [0.82, 1.90, 1.11]]])


Diagonal Matrix
return the vector of value along the diagonal of a matrix

In [23]:
# Diagonal matrix return only the diagonal value of a matrix

T = torch.rand(3,3)

cein = torch.einsum('ii->i',T)
print(f'T: {T} \nT shape: {T.shape}')
c_diag = torch.diag(T)

print(f'regular: {c_diag} \n  eisum: {cein}')


T: tensor([[0.45, 0.36, 0.29],
        [0.64, 0.86, 0.76],
        [0.19, 0.17, 0.62]]) 
T shape: torch.Size([3, 3])
regular: tensor([0.45, 0.86, 0.62]) 
  eisum: tensor([0.45, 0.86, 0.62])


In [24]:
# torch.einsum('ii->i',T)

Trace

Take the sum of all values along the diagonal axix of a matrix

In [25]:
# matrix trace

c_trace = torch.trace(T)
cein = torch.einsum('ii->',T)
print(f'T: {T}')
print(f'regular: {c_trace} \n  eisum: {cein}')

T: tensor([[0.45, 0.36, 0.29],
        [0.64, 0.86, 0.76],
        [0.19, 0.17, 0.62]])
regular: 1.9370249509811401 
  eisum: 1.9370249509811401


Tensor Contraction  
Shrinking the dimension of the tensor  
must provide the dimension to be ignored


In [26]:
o = torch.rand((3,4,2))
p = torch.rand((4,3,6))
print(f'value: {o.shape} value2: {p.shape}')

c_tdot = torch.tensordot(o,p,dims=([1,0],[0,1]))
cein = torch.einsum('ijk,jil->kl',o,p)
print(f'regular: {c_tdot} \n  eisum: {cein}')

value: torch.Size([3, 4, 2]) value2: torch.Size([4, 3, 6])
regular: tensor([[2.85, 2.90, 2.53, 3.08, 2.98, 3.10],
        [2.32, 2.17, 2.23, 2.37, 2.47, 2.86]]) 
  eisum: tensor([[2.85, 2.90, 2.53, 3.08, 2.98, 3.10],
        [2.32, 2.17, 2.23, 2.37, 2.47, 2.86]])
